In [9]:
import pandas as pd
import ast
from funciones import extract

In [10]:
df = pd.read_csv(r'data_set_limpio/games_preparado.csv.gz')

In [12]:
item = pd.read_csv(r'data_set_limpio/items_preparado.csv.gz')

In [14]:
opinion = pd.read_csv(r'data_set_limpio/reviews_preparado.csv.gz')

In [15]:
def developer(x):
    
    fecha_inicio = df.loc[df['developer'] == x]['release_date'].min()[:4]
    fecha_final =  df.loc[df['developer'] == x]['release_date'].max()
    
    anio = {}
    free = {}
    
    while fecha_inicio <= fecha_final:
        if len(df[(df['release_date'] >= fecha_inicio) & (df['release_date'] <= fecha_inicio[:4]+'-12-31') & (df['developer'] == x)]) > 0:
            anio[fecha_inicio[:4]] = len(df[(df['release_date'] >= fecha_inicio) & (df['release_date'] <= fecha_inicio[:4]+'-12-31') & (df['developer'] == x)])
            free[fecha_inicio[:4]] = len(df[(df['release_date'] >= fecha_inicio) & (df['release_date'] <= fecha_inicio[:4]+'-12-31') & (df['developer'] == x) & ((df['price'] == 'Free') | (df['price'] == 'Free to Play') )])
        fecha_inicio = str(int(fecha_inicio[:4])+1)+'-01-01'

    for x,y  in free.items():
        free[x] = str(round((y / anio[x])*100,2))+'%'
        
    resultado = {'Cantidad de Items': anio,'Contenido Free':free}
    
    return resultado




In [16]:
developer('Laush Dmitriy Sergeevich')

{'Cantidad de Items': {'2017': 9, '2018': 1},
 'Contenido Free': {'2017': '0.0%', '2018': '0.0%'}}

In [29]:
def userdata(x):
    
    precios = []
    respuesta = {}
    
    usuario = item.loc[item['user_id'] == x]['items']
    if not usuario.empty:
        usuario = usuario.iloc[0]
    data = ast.literal_eval(usuario)
    result = pd.DataFrame(data)
    result.dropna(inplace=True)

    for y in result['item_name']:
        price = df.loc[df['app_name'] == y]['price'].values
        if len(price) > 0:
            try: 
                price_value = float(price[0])
                precios.append(price_value)
            except ValueError:
                pass  
    
    respuesta['Usuario'] = x
    respuesta['Dinero gastado'] = str(round(sum(precios)))+' USD'
    respuesta["cantidad de items"] = item.loc[item['user_id'] == x]['items_count'].values[0]
    
    coment = len(opinion.loc[opinion['user'] == x])
    
    respuesta['Porcentaje de recomendaciones'] = str(round(((len(opinion.loc[opinion['user'] == x]) / item.loc[item['user_id'] == x]['items_count'].values[0])) * 100,2))+ '%'
    
    
    return respuesta




In [30]:
userdata('js41637')

{'Usuario': 'js41637',
 'Dinero gastado': '5847 USD',
 'cantidad de items': 888,
 'Porcentaje de recomendaciones': '0.34%'}

In [31]:
usuario = item[['user_id','items']][:1].values
usuario = dict(usuario)

In [31]:
item

,user_id,items_count,steam_id,user_url,items,item_max_plating,item_max_hour
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",Counter-Strike: Global Offensive,23532
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...",The Elder Scrolls V: Skyrim,13831
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...",Idling to Rule the Gods,28545
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...",Grand Theft Auto IV,52062
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...",Counter-Strike: Global Offensive,19800
...,...,...,...,...,...,...,...
71499,76561198320136420,321,76561198320136420,http://steamcommunity.com/profiles/76561198320...,"[{'item_id': '11020', 'item_name': 'TrackMania...",Evolve Stage 2,103
71500,ArkPlays7,4,76561198321542621,http://steamcommunity.com/id/ArkPlays7,"[{'item_id': '218230', 'item_name': 'PlanetSid...",Counter-Strike: Global Offensive,4557
71501,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla...",Emily is Away,8
71502,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania...",TrackMania Nations Forever,0


In [42]:

mayor = {'genero': '','usuario' : '','juego' : '','anio' : '','horas' : ''}


for x,y in usuario.items():
    
    for e in filtro:

        data = ast.literal_eval(y)
        result = pd.DataFrame(data)
        result.dropna(inplace=True)
        result = pd.merge(result, df[['app_name',e,'release_date']], left_on='item_name', right_on='app_name')


        parafiltro = result.loc[result[e] == 1]


        max = parafiltro.loc[parafiltro['playtime_forever'] == parafiltro['playtime_forever'].max()]

        


mayor

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:


data = ast.literal_eval(y)
result = pd.DataFrame(data)
result.dropna(inplace=True)
result = pd.merge(result, df[['app_name',e,'release_date']], left_on='item_name', right_on='app_name')


parafiltro = result.loc[result[e] == 1]


max = parafiltro.loc[parafiltro['playtime_forever'] == parafiltro['playtime_forever'].max()]